# Моделирование тяжести и последствий ДТП

## 1. Постановка задач

В этом разделе моделируются различные аспекты тяжести ДТП:

- Модель 1: предсказание числа погибших с использованием счётных регрессионных моделей
- Модель 2: классификация ДТП на случаи с погибшими и без
- Модель 3: оценка экономического ущерба от ДТП и факторов, его определяющих

Цель — понять, какие факторы влияют на последствия ДТП и как можно снизить тяжесть аварий с точки зрения смертности и ущерба.

---

## 2. Подготовка данных

- Загрузка необходимых библиотек
- Объединение таблиц `accidents`, `participants`, `vehicles`
- Фильтрация аварий с валидными значениями ключевых переменных
- Создание новых признаков:
  - `is_fatal` — бинарный индикатор наличия погибших
  - `log_cost` — логарифм экономического ущерба
- Разделение на тренировочную и тестовую выборки (например, 80/20)

In [1]:
# Загрузка библиотек и утилит
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("../scripts")) # Добавляем путь к скриптам
from utils import (
    load_full_data_to_sqlite,
    run_query,
    save_png,
    save_html
)

# Пути для сохранения визуализаций
FIG_DIR = "../outputs/figures"
FIG_INT_DIR = "../outputs/figures/interactive"

In [2]:
def load_full_data_to_sqlite(acc_url, part_url, veh_url, db_path):
    import sqlite3, pandas as pd, requests, time
    conn = sqlite3.connect(db_path)
    for public_url, table in [(acc_url, 'accidents'),
                              (part_url, 'participants'),
                              (veh_url, 'vehicles')]:
        print(f"Processing {table}...")
        start = time.time()
        href = requests.get(
            "https://cloud-api.yandex.net/v1/disk/public/resources/download",
            params={"public_key": public_url}
        ).json()["href"]
        print(f"Got href for {table} in {time.time() - start:.2f}s")
        start = time.time()
        df = pd.read_csv(href, sep=';')
        print(f"Read CSV for {table} in {time.time() - start:.2f}s")
        start = time.time()
        df.to_sql(table, conn, if_exists='replace', index=False)
        print(f"Saved {table} to SQLite in {time.time() - start:.2f}s")
    return conn

### Загрузка данных в SQLite

In [5]:
accidents_url    = "https://disk.yandex.ru/d/yPdgwafR_2xElg"
participants_url = "https://disk.yandex.ru/d/YeyKLfXuETaEUQ"
vehicles_url     = "https://disk.yandex.ru/d/NJApFGWb85CWVQ"

conn = load_full_data_to_sqlite(
    accidents_url,
    participants_url,
    vehicles_url,
    db_path = "../data/crash_database.db"
)

Processing accidents...
Got href for accidents in 11.65s
Read CSV for accidents in 63.18s
Saved accidents to SQLite in 169.97s
Processing participants...
Got href for participants in 11.64s
Read CSV for participants in 81.24s
Saved participants to SQLite in 105.95s
Processing vehicles...
Got href for vehicles in 0.63s
Read CSV for vehicles in 35.80s
Saved vehicles to SQLite in 23.81s


### SQL-запрос с объединением таблиц

Для анализа факторов, влияющих на аварийность, разработан SQL-запрос, объединяющий данные из таблиц `accidents`, `participants` и `vehicles`. Основная цель — собрать релевантные переменные, исключив неинформативные или сложные для интерпретации данные, чтобы обеспечить чистоту и воспроизводимость анализа.

**Выбранные переменные**

- **Таблица `accidents`**:
  - `id`: уникальный идентификатор ДТП
  - `category`: тип ДТП
  - `datetime`: дата и время происшествия
  - `light`: состояние освещения
  - `participants_count`: количество участников
  - `dead_count`: количество погибших
  - `injured_count`: количество пострадавших
  - `weather`: погодные условия
- **Таблица `participants`**:
  - `accident_id`: ключ для связи с таблицей `accidents`
  - `role`: роль участника (водитель, пешеход и т.д.)
  - `gender`: пол участника
  - `violations`: нарушение ПДД
  - `years_of_driving_experience`: стаж вождения
- **Таблица `vehicles`**:
  - `year`: год производства ТС

**Переменные, исключённые из анализа**

Для повышения качества модели и упрощения интерпретации результатов исключены следующие данные:
- **Пространственные характеристики** (`region`, `county`, `longitude`, `latitude`): не входят в задачи проекта, так как фокус не на геоанализе.
- **Сложные текстовые переменные** (`nearby`, `road_conditions`, `participant_categories`): трудно интерпретировать из-за неоднозначности и отсутствия стандартизации.
- **Категориальные переменные с высокой кардинальностью** (`brand`, `vehicles.category`, `model`): большое число уникальных значений затрудняет их использование в моделировании.
- **Недостаточно документированные данные** (`severity`): отсутствие четкого определения "тяжести" ДТП в документации ГИБДД.
- **Последствия, а не причины** (`health_status`): зависят от самого ДТП и не могут использоваться как предикторы.
- **Технические артефакты**: лишние идентификаторы и служебные ключи, не участвующие в моделировании.

**Примечания**

- Запрос оптимизирован для отбора только релевантных признаков с целью снижения размерности и повышения читаемости модели.
- Исключение категориальных переменных с высокой кардинальностью и недокументированных данных повышает воспроизводимость и интерпретируемость результатов.

**Особые условия объединения**

Так как в одном ДТП может быть несколько участников (в том числе водителей), применена гибкая логика агрегации:
- Если в ДТП указан **один водитель-нарушитель** — анализ строится на его характеристиках.
- При нескольких **водителях-нарушителях** — признаки агрегируются (например, минимальный стаж, наличие женщин).
- При **отсутствии нарушителей (всего 1.2% ДТП)** — признаки агрегируются по всем водителям.

Признаки **агрегируются** на уровне ДТП таким образом, чтобы учесть состав участников и траснпортных средств без искажения выборки. Для отдельных переменных данные агрегируются следующим образом:

- Стаж вождения: среднее значение по выбранным водителям
- Пол: категориальный признак с тремя значениями: `all_male`, `all_female`, `mixed`
- Возраст транспортного средства: усреднённый по всем задействованным ТС, принадлежащих выбранным водителям. Возраст вычисляется как разница между годом ДТП `strftime('%Y', a.datetime)` и годом производства автомобиля `v.year`
- Наличие нарушителей-пешеходов: бинарный индикатор (true/false)

In [51]:
# Генерируем SQL-запрос как строку на Python

sql_query = """
WITH drivers AS (
    SELECT *
    FROM participants
    WHERE role = 'Водитель'
),

violating_drivers AS (
    SELECT *
    FROM drivers
    WHERE violations IS NOT NULL AND TRIM(violations) != ''
),

selected_drivers AS (
    SELECT accident_id, participant_id, gender, years_of_driving_experience,
           CASE
               WHEN COUNT(*) = 1 THEN 'single_violator'
               ELSE 'multiple_violators'
           END AS violator_case
    FROM violating_drivers
    GROUP BY accident_id
),

fallback_drivers AS (
    SELECT accident_id, participant_id, gender, years_of_driving_experience,
           'no_violator' AS violator_case
    FROM drivers
    WHERE accident_id NOT IN (SELECT DISTINCT accident_id FROM violating_drivers)
),

final_drivers AS (
    SELECT * FROM selected_drivers
    UNION ALL
    SELECT * FROM fallback_drivers
),

driver_vehicles AS (
    SELECT d.accident_id, p.vehicle_id
    FROM final_drivers d
    JOIN participants p ON d.participant_id = p.participant_id
    WHERE p.vehicle_id IS NOT NULL
),

vehicle_ages AS (
    SELECT dv.accident_id,
           AVG(CAST(strftime('%Y', a.datetime) AS INTEGER) - v.year) AS avg_vehicle_age
    FROM driver_vehicles dv
    JOIN vehicles v ON dv.vehicle_id = v.vehicle_id
    JOIN accidents a ON a.id = dv.accident_id
    GROUP BY dv.accident_id
),

driver_aggregates AS (
    SELECT accident_id,
           COUNT(*) AS driver_count,
           AVG(years_of_driving_experience) AS avg_experience,
           CASE
               WHEN SUM(CASE WHEN gender = 'Женский' THEN 1 ELSE 0 END) = 0 THEN 'all_male'
               WHEN SUM(CASE WHEN gender = 'Мужской' THEN 1 ELSE 0 END) = 0 THEN 'all_female'
               ELSE 'mixed'
           END AS drivers_gender
    FROM final_drivers
    GROUP BY accident_id
),

pedestrian_flag AS (
    SELECT accident_id,
           MAX(CASE WHEN role = 'Пешеход' AND violations IS NOT NULL AND TRIM(violations) != '' THEN 1 ELSE 0 END) AS has_violating_pedestrian
    FROM participants
    GROUP BY accident_id
)

SELECT 
    da.accident_id,
    a.category,
    a.datetime,
    a.light,
    a.weather,
    a.participants_count,
    a.dead_count,
    a.injured_count,
    da.driver_count,
    da.avg_experience,
    da.drivers_gender,
    va.avg_vehicle_age,
    pf.has_violating_pedestrian
FROM driver_aggregates AS da
JOIN accidents AS a ON da.accident_id = a.id  -- Добавляем JOIN с accidents
LEFT JOIN vehicle_ages AS va ON da.accident_id = va.accident_id
LEFT JOIN pedestrian_flag AS pf ON da.accident_id = pf.accident_id;
"""

# Читаем результат в DataFrame
df = pd.read_sql_query(sql_query, conn)

# Проверим
print(df.head())
print(df.info())

   accident_id           category             datetime                light  \
0      2869171  Наезд на пешехода  2023-05-24 19:30:00  Светлое время суток   
1      2320091       Иной вид ДТП  2015-09-16 13:00:00  Светлое время суток   
2      2321011  Падение пассажира  2017-08-25 15:30:00  Светлое время суток   
3      2576909       Столкновение  2021-05-29 19:40:00  Светлое время суток   
4      2547743       Столкновение  2021-02-22 13:50:00  Светлое время суток   

    weather  participants_count  dead_count  injured_count  driver_count  \
0      Ясно                   2           0              1             1   
1  Пасмурно                   2           0              1             1   
2      Ясно                   3           0              1             1   
3      Ясно                   3           0              1             1   
4      Ясно                   4           0              3             1   

   avg_experience drivers_gender  avg_vehicle_age  has_violating_ped

### Фильтрация и генерация признаков

Для успешного моделирования и анализа данных о ДТП проведены преобразования переменных с целью улучшения качества модели, уменьшения кардинальности и повышения интерпретируемости. Приведенные ниже преобразования способствуют учету важных факторов, влияющих на тяжесть ДТП, с сохранением логической связи с реальными условиями.

#### 1. Тип ДТП (category)
Переменная `category` имеет 18 уникальных значений, что создает избыточность для анализа, а также многие категории представлены малым числом наблюдений. Для уменьшения кардинальности выбраны топ-7 категорий, составляющие более 95% всех ДТП. Оставшиеся категории объединены в категорию "Другой тип".

#### 2. Условия освещения (light)
Переменная `light` имеет 6 категорий, однако категория "не установлено" содержит лишь 68 наблюдений и была удалена. Также были объединены категории "В темное время суток, освещение отсутствует" и "В темное время суток, освещение не включено", поскольку они имеют схожие характеристики.

#### 3. Дата и время ДТП (datetime)
Переменная `datetime` играет важную роль, так как интенсивность движения изменяется в зависимости от времени суток и дня недели. Время имеет косвенное влияние на количество ДТП через другие факторы (например, внимание водителей).

##### Дата:
Для анализа были созданы две переменные:
- Категориальная переменная для каждого дня недели как отдельной категории
- Бинарная переменная, разделяющая дни на будние и выходные

##### Время:
Для времени выбраны два подхода кодирования:
- **Синусо-косинусная трансформация времени** (стандартный метод): преобразование в синусоидальные значения через формулы `sin(2π·hour/24)` и `cos(2π·hour/24)`
- **Категориальное разбиение времени на периоды**: дни разделены на несколько категорий в зависимости от интенсивности движения, как показано в распределении ДТП:
  - 7-8 утра — утренний час пик
  - 9-16 — дневная активность
  - 17-20 — вечерний час пик
  - 21-6 утра — ночной спад

Выберется та переменная, которая даст наилучший результат в модели.

#### 4. Погода (weather)
Переменная `weather` содержит 97% значений с категориями "ясно", "пасмурно", "дождь", "снегопад". Остальные значения представляют редкие или смешанные состояния. Мы предполагаем, что "ясно" и "пасмурно" не сильно различаются по влиянию на аварийность, поэтому было принято решение разделить условия на **благоприятные** и **неблагоприятные**. 
- Неблагоприятные условия включают туман, дождь, снегопад, метель и ураганный ветер. Создана бинарная переменная, которая принимает значение 1, если в значении погоды присутствует хотя бы одно из перечисленных состояний, и 0 в противном случае.

#### 5. Количество погибших (dead_count) и летальность (lethality)
Переменная `dead_count` сохраняется в данных, а также добавляется бинарная переменная `lethality`, которая принимает значение 1, если в ДТП есть погибшие, и 0, если их нет.

#### 6. Обработка пропущенных значений
Много пропущенных значений содержат переменные `avg_experience` и `ave_vehicle_age`. Пропущенные значения заменены на медианные для соответствующих переменных.

#### 7. Удаленные переменные
Переменная `driver_count` (число водителей) была удалена, так как является технической и не имеет значительного влияния на моделирование.

In [52]:
import pandas as pd
import numpy as np

# Создаем новый DataFrame df_processed для обработанных данных
df_processed = df.copy()

# 1. Преобразование переменной `category`
# Оставляем топ-7 категорий, остальные объединяем в 'Другой тип'
top_7_categories = df_processed['category'].value_counts().head(7).index
df_processed['category'] = df_processed['category'].apply(lambda x: x if x in top_7_categories else 'Другой тип')

# 2. Преобразование переменной `light`
# Удаляем категорию 'не установлено' и объединяем схожие категории
df_processed = df_processed[df_processed['light'] != 'не установлено']  # Удаляем 'не установлено'

df_processed['light'] = df_processed['light'].replace({
    'В темное время суток, освещение не включено': 'В темное время суток, освещение отсутствует'
})

# 3. Преобразование переменной `datetime`
# Убедимся, что datetime в правильном формате
df_processed['datetime'] = pd.to_datetime(df_processed['datetime'])

# 3.1. Дата: день недели и будни/выходные
df_processed['day_of_week'] = df_processed['datetime'].dt.day_name()  # Категориальная переменная: Monday, Tuesday, ...
df_processed['is_weekend'] = df_processed['datetime'].dt.weekday >= 5  # True (1) для выходных (сб, вс), False (0) для будней

# 3.2. Время: синусо-косинусная трансформация
df_processed['hour'] = df_processed['datetime'].dt.hour
df_processed['sin_time'] = np.sin(2 * np.pi * df_processed['hour'] / 24)
df_processed['cos_time'] = np.cos(2 * np.pi * df_processed['hour'] / 24)

# 3.3. Время: категориальное разбиение на периоды
def time_period(hour):
    if 7 <= hour <= 8:
        return 'утренний час пик'
    elif 9 <= hour <= 16:
        return 'дневная активность'
    elif 17 <= hour <= 20:
        return 'вечерний час пик'
    else:
        return 'ночной спад'

df_processed['time_period'] = df_processed['hour'].apply(time_period)

# 4. Преобразование переменной `weather`
# Создаем бинарную переменную: 1 для неблагоприятных условий, 0 для благоприятных
adverse_conditions = ['дождь', 'снегопад', 'метель', 'ураганный ветер', 'туман']
df_processed['adverse_weather'] = df_processed['weather'].apply(
    lambda x: 1 if any(cond.lower() in str(x).lower() for cond in adverse_conditions) else 0
)

# 5. Преобразование `dead_count` и создание `lethality`
# Оставляем dead_count как есть, добавляем бинарную переменную lethality
df_processed['lethality'] = df_processed['dead_count'].apply(lambda x: 1 if x > 0 else 0)

# 6. Замена пропусков медианой
df_processed['avg_experience'] = df_processed['avg_experience'].fillna(df_processed['avg_experience'].median())
df_processed['avg_vehicle_age'] = df_processed['avg_vehicle_age'].fillna(df_processed['avg_vehicle_age'].median())

# 7. Удаление переменной `driver_count`
df_processed = df_processed.drop(columns=['driver_count'])

# 8. Преобразование булевых переменных в числовые
df_processed['is_weekend'] = df_processed['is_weekend'].astype(int)

#### Проверка целостности данных

После обработки данных в `df_processed` пропущенные значения отсутствуют

#### Обработка категориальных переменных

Для дальнейшего анализа необходимо преобразовать категориальные переменные в числовые. В зависимости от модели применяются различные способы кодирования:

**Регрессионные модели** 
- One-hot encoding (для каждой категории создается отдельная бинарная переменная) с удалением переменной базового (референсного) уровня, что помогает избежать мультиколлинеарности. Такой подход позволяет сохранить интерпретируемость модели.
- Создается `df_regression` при помощи библиотеки **`sklearn`**
- Референсный уровень выбирается вручную для наглялности интерпретации 

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Спискок категориальных переменных
categorical_columns = ['category', 'light', 'day_of_week', 'drivers_gender', 'time_period']

# Выбор референсных уровней
drop_categories = {
    'category': 'Другой тип',  
    'light': 'Не установлено',  
    'day_of_week': 'Monday',
    'time_period': 'ночной спад',
    'drivers_gender': 'all_male'
}

# Настройка ColumnTransformer с полным кодированием
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False), categorical_columns)
    ],
    remainder='passthrough'  # Сохранение числовых и бинарных столбцов
)

# Применение трансформации
df_regression = pd.DataFrame(
    preprocessor.fit_transform(df_processed),
    columns=preprocessor.get_feature_names_out()
)

# Ручное удаление выбранных категорий
for col, drop_cat in drop_categories.items():
    drop_col = f'cat__{col}_{drop_cat}'
    if drop_col in df_regression.columns:
        df_regression = df_regression.drop(columns=[drop_col])
    else:
        print(f"Предупреждение: {drop_col} не найдена в закодированных столбцах!")

# Приводим все необходимые колонки к числовому типу
df_regression = df_regression.apply(lambda col: pd.to_numeric(col, errors='coerce'))

# Удаляем лишние
df_regression = df_regression.drop(columns=df_regression.select_dtypes(include='object').columns)

df_regression = df_regression.drop(columns=[
    'remainder__lethality',
    'remainder__injured_count',
    'remainder__weather',
    'remainder__datetime'
])

## 3. Модель 1: Счётные модели для предсказания числа погибших

### 3.1. Обоснование выбора модели

Целью модели является предсказание числа погибших, так как основным приоритетом в политике по снижению аварийности является минимизация числа смертельных случаев. Число погибших является **дискретной счётной переменной** (`dead_count ∈ {0,1,2,...}`) с преобладанием нулевых значений, что делает её подходящей для применения **счётных моделей** — пуассоновской и негативной биномиальной регрессий.

Счётные модели являются стандартным инструментом в современных академических исследованиях факторов аварийности. Они обеспечивают чёткую интерпретацию влияния различных факторов на зависимую переменную, демонстрируя, как изменение значений предикторов сказывается на числе погибших.

Мы рассматриваем **семейство счётных моделей**, так как они включают несколько типов моделей:

- **Пуассоновская регрессия** (Poisson regression) предполагает, что среднее значение переменной равно её дисперсии.
- Если наблюдается **overdispersion** (дисперсия > среднее), применяется **негативная биномиальная регрессия**.
- В случае с **избыточными нулями** (когда много аварий без погибших) могут быть использованы **Zero-Inflated** модификации соответствующих моделей, где нулевые и ненулевые значения моделируются независимо.

Для более подробного знакомства с применением счётных моделей в анализе аварийности см. статью: [Road Crash Prediction Models: Different Statistical Modeling Approaches](https://doi.org/10.4236/jtts.2017.72014).

Для выбора подходящей модели рассчитаны среднее и дисперсия `remainder__dead_count`.

In [21]:
# Расчет среднего и дисперсии для dead_count

mean_dead_count = df_regression["remainder__dead_count"].mean()
variance_dead_count = df_regression["remainder__dead_count"].var()

In [37]:
print(df_regression.isna().sum())

cat__category_Наезд на велосипедиста                      0
cat__category_Наезд на пешехода                           0
cat__category_Наезд на препятствие                        0
cat__category_Опрокидывание                               0
cat__category_Падение пассажира                           0
cat__category_Столкновение                                0
cat__category_Съезд с дороги                              0
cat__light_В темное время суток, освещение включено       0
cat__light_В темное время суток, освещение отсутствует    0
cat__light_Светлое время суток                            0
cat__light_Сумерки                                        0
cat__day_of_week_Friday                                   0
cat__day_of_week_Saturday                                 0
cat__day_of_week_Sunday                                   0
cat__day_of_week_Thursday                                 0
cat__day_of_week_Tuesday                                  0
cat__day_of_week_Wednesday              

#### Выбор модели

- Дисперсия (0.15) превышает среднее (0.11), что указывает на **overdispersion**. Основной моделью выбрана негативная биномиальная регрессия. Для сравнения дополнительно используется пуассоновская регрессия.

- Учитывая избыток нулевых значений (соотношение аварий без погибших к авариям с погибшими 9:1), применяются **Zero-Inflated** модификации обеих моделей.

### 3.2. Описание переменных

#### Зависимая переменная
- `remainder__dead_count`: Число погибших в ДТП (дискретная счётная переменная).

#### Предикторы
- **Категориальные переменные (закодированы через One-Hot Encoding с ручным удалением референсных уровней):**
  - `cat__category_*`:
    - `cat__category_Наезд на велосипедиста`: 1, если ДТП связано с наездом на велосипедиста, 0 — иначе.
    - `cat__category_Наезд на пешехода`: 1, если ДТП связано с наездом на пешехода, 0 — иначе.
    - `cat__category_Наезд на препятствие`: 1, если ДТП связано с наездом на препятствие, 0 — иначе.
    - `cat__category_Опрокидывание`: 1, если ДТП связано с опрокидыванием, 0 — иначе.
    - `cat__category_Падение пассажира`: 1, если ДТП связано с падением пассажира, 0 — иначе.
    - `cat__category_Столкновение`: 1, если ДТП связано со столкновением, 0 — иначе.
    - `cat__category_Съезд с дороги`: 1, если ДТП связано со съездом с дороги, 0 — иначе.
    - *Референсный уровень*: `cat__category_Другой тип` (удалён).
  - `cat__light_*`:
    - `cat__light_В темное время суток, освещение включено`: 1, если ДТП произошло в тёмное время суток с включённым освещением, 0 — иначе.
    - `cat__light_В темное время суток, освещение отсутствует`: 1, если ДТП произошло в тёмное время суток без освещения, 0 — иначе.
    - `cat__light_Светлое время суток`: 1, если ДТП произошло в светлое время суток, 0 — иначе.
    - `cat__light_Сумерки`: 1, если ДТП произошло в сумерках, 0 — иначе.
    - *Референсный уровень*: `cat__light_Не установлено` (удалён).
  - `cat__day_of_week_*`:
    - `cat__day_of_week_Friday`: 1, если ДТП произошло в пятницу, 0 — иначе.
    - `cat__day_of_week_Saturday`: 1, если ДТП произошло в субботу, 0 — иначе.
    - `cat__day_of_week_Sunday`: 1, если ДТП произошло в воскресенье, 0 — иначе.
    - `cat__day_of_week_Thursday`: 1, если ДТП произошло в четверг, 0 — иначе.
    - `cat__day_of_week_Tuesday`: 1, если ДТП произошло во вторник, 0 — иначе.
    - `cat__day_of_week_Wednesday`: 1, если ДТП произошло в среду, 0 — иначе.
    - *Референсный уровень*: `cat__day_of_week_Monday` (удалён).
  - `cat__drivers_gender_*`:
    - `cat__drivers_gender_all_female`: 1, если все водители — женщины, 0 — иначе.
    - `cat__drivers_gender_mixed`: 1, если состав водителей смешанный, 0 — иначе.
    - *Референсный уровень*: `cat__drivers_gender_all_male` (удалён).
  - `cat__time_period_*`:
    - `cat__time_period_вечерний час пик`: 1, если ДТП произошло в вечерний час пик, 0 — иначе.
    - `cat__time_period_дневная активность`: 1, если ДТП произошло в период дневной активности, 0 — иначе.
    - `cat__time_period_утренний час пик`: 1, если ДТП произошло в утренний час пик, 0 — иначе.
    - *Референсный уровень*: `cat__time_period_ночной спад` (удалён).

- **Числовые и бинарные переменные:**
  - `remainder__accident_id`: Уникальный идентификатор ДТП (числовой).
  - `remainder__avg_experience`: Средний стаж водителей (числовой).
  - `remainder__avg_vehicle_age`: Средний возраст транспортных средств (числовой).
  - `remainder__has_violating_pedestrian`: 1, если в ДТП участвовал нарушающий правила пешеход, 0 — иначе.
  - `remainder__is_weekend`: 1, если ДТП произошло в выходной день, 0 — иначе.
  - `remainder__sin_time`: Синус времени суток (циклическое представление времени, числовой).
  - `remainder__cos_time`: Косинус времени суток (циклическое представление времени, числовой).
  - `remainder__adverse_weather`: 1, если погодные условия неблагоприятные, 0 — иначе.
  - `remainder__participants_count`: Число участников ДТП (числовой, используется как переменная экспозиции).

### 3.3. Применение и сравнение моделей

#### Инструменты

Для реализации **Zero-Inflated** моделей использована библиотека **`statsmodels`**, поскольку **`sklearn`** не поддерживает такие модели напрямую

#### Этапы моделирования

 1. **Сравнение Zero-Inflated Negative Binomial (ZINB) моделей:**

- Построены 4 ZINB модели с различными комбинациями временных и дневных признаков:
  
    - Комбинация 1: Дни недели + циклическое представление времени.
      
    - Комбинация 2: Дни недели + временные периоды.
      
    - Комбинация 3: Выходные vs будние дни + циклическое представление времени.
      
    - Комбинация 4: Выходные vs будние дни + временные периоды.

Для оценки качества использована метрика Mean Squared Error (MSE), стандартная для задач регрессии, включая счётные модели. MSE измеряет среднеквадратичное отклонение прогнозов от реальных значений, что позволяет выбрать модель с наименьшей ошибкой.

In [54]:
# Выбираем только числовые переменные (уже приведённые к float)
df_num = df_regression.select_dtypes(include=['float', 'int'])

# 1. Общая статистика
print("📊 Общая статистика:")
display(df_num.describe().T.sort_values('std', ascending=False))

# 2. Сколько уникальных значений в каждой колонке
print("\n🔢 Уникальные значения:")
unique_counts = df_num.nunique().sort_values()
display(unique_counts)

# 3. Сколько нулей и сколько единиц в бинарных переменных
print("\n⚠️ Частоты значений 0 и 1 (для бинарных):")
for col in df_num.columns:
    vals = df_num[col].value_counts()
    if len(vals) <= 3:  # фильтруем по дискретности
        print(f"{col:50s} | ", end="")
        for v in sorted(vals.index):
            print(f"{v}: {vals[v]}", end="  ")
        print()

# 4. Проверка на признаки с константным значением
print("\n🟨 Признаки с одинаковым значением во всей колонке:")
constant_cols = df_num.columns[df_num.nunique() <= 1]
print(constant_cols.tolist())

# 5. Проверка на большие значения
print("\n🔥 Признаки с потенциально взрывающим масштабом (max > 1000):")
for col in df_num.columns:
    max_val = df_num[col].max()
    if max_val > 1000:
        print(f"{col:50s} | max = {max_val}")

# 6. Корреляция между признаками
print("\n📈 Высокая корреляция между признаками (r > 0.95):")
corr = df_num.corr().abs()
high_corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
high_corr_pairs = high_corr.stack()[high_corr.stack() > 0.95]
print(high_corr_pairs)


📊 Общая статистика:


,count,mean,std,min,25%,50%,75%,max
remainder__accident_id,1427609.0,2.250597e+06,417875.375613,1527476.0,1.888124e+06,2.250554e+06,2.611424e+06,2978754.0
remainder__avg_experience,1427609.0,1.424339e+01,10.267505,1.0,7.000000e+00,1.200000e+01,1.900000e+01,73.0
remainder__avg_vehicle_age,1427609.0,1.117665e+01,9.261537,0.0,5.000000e+00,1.000000e+01,1.500000e+01,2014.0
remainder__hour,1427609.0,1.380404e+01,5.784188,0.0,1.000000e+01,1.500000e+01,1.800000e+01,23.0
remainder__participants_count,1427609.0,2.475969e+00,1.185318,1.0,2.000000e+00,2.000000e+00,3.000000e+00,59.0
remainder__sin_time,1427609.0,-2.444430e-01,0.687098,-1.0,-8.660254e-01,-5.000000e-01,2.588190e-01,1.0
remainder__cos_time,1427609.0,-1.817184e-01,0.659638,-1.0,-8.660254e-01,-2.588190e-01,5.000000e-01,1.0
cat__category_Столкновение,1427609.0,4.300092e-01,0.495077,0.0,0.000000e+00,0.000000e+00,1.000000e+00,1.0
cat__time_period_дневная активность,1427609.0,4.154646e-01,0.492802,0.0,0.000000e+00,0.000000e+00,1.000000e+00,1.0
cat__light_Светлое время суток,1427609.0,6.287366e-01,0.483143,0.0,0.000000e+00,1.000000e+00,1.000000e+00,1.0



🔢 Уникальные значения:


cat__category_Наезд на велосипедиста                            2
remainder__is_weekend                                           2
remainder__has_violating_pedestrian                             2
cat__time_period_утренний час пик                               2
cat__time_period_дневная активность                             2
cat__time_period_вечерний час пик                               2
cat__drivers_gender_mixed                                       2
cat__drivers_gender_all_female                                  2
cat__day_of_week_Tuesday                                        2
cat__day_of_week_Thursday                                       2
cat__day_of_week_Sunday                                         2
cat__day_of_week_Saturday                                       2
cat__day_of_week_Wednesday                                      2
cat__light_Сумерки                                              2
cat__category_Наезд на пешехода                                 2
cat__categ


⚠️ Частоты значений 0 и 1 (для бинарных):
cat__category_Наезд на велосипедиста               | 0.0: 1380545  1.0: 47064  
cat__category_Наезд на пешехода                    | 0.0: 1025046  1.0: 402563  
cat__category_Наезд на препятствие                 | 0.0: 1341902  1.0: 85707  
cat__category_Опрокидывание                        | 0.0: 1340354  1.0: 87255  
cat__category_Падение пассажира                    | 0.0: 1382247  1.0: 45362  
cat__category_Столкновение                         | 0.0: 813724  1.0: 613885  
cat__category_Съезд с дороги                       | 0.0: 1340641  1.0: 86968  
cat__light_В темное время суток, освещение включено | 0.0: 1112850  1.0: 314759  
cat__light_В темное время суток, освещение отсутствует | 0.0: 1252328  1.0: 175281  
cat__light_Светлое время суток                     | 0.0: 530019  1.0: 897590  
cat__light_Сумерки                                 | 0.0: 1387698  1.0: 39911  
cat__day_of_week_Friday                            | 0.0: 1206661  1.

In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from time import time

# Целевая переменная и экспозиция
y = df_regression['remainder__dead_count'].astype(float)
exposure = df_regression['remainder__participants_count'].astype(float)

# Базовое ядро признаков (которые точно работают)
core_features = [
    'remainder__sin_time', 'remainder__cos_time',
    'cat__day_of_week_Friday', 'cat__day_of_week_Saturday',
    'cat__category_Наезд на пешехода'
]

# Кандидаты на добавление по одному
candidates = [
    'cat__category_Опрокидывание',
    'cat__category_Столкновение',
    'cat__light_Сумерки',
    'cat__light_В темное время суток, освещение включено',
    'cat__light_В темное время суток, освещение отсутствует',
    'cat__drivers_gender_mixed',
    'remainder__avg_vehicle_age',
    'remainder__avg_experience',
    'remainder__has_violating_pedestrian',
    'cat__time_period_утренний час пик',
    'cat__time_period_вечерний час пик',
    'remainder__adverse_weather'
]

# Запуск перебора
print("🔍 Начинаем поочерёдное добавление признаков к ядру...\n")
for feat in candidates:
    features = core_features + [feat]

    try:
        # Подготовка X
        X = df_regression[features].astype(float)

        # Сплит
        X_train, X_test, y_train, y_test, exp_train, exp_test = train_test_split(
            X, y, exposure, test_size=0.2, random_state=42
        )

        # Добавляем константу
        X_train_sm = sm.add_constant(X_train)
        X_test_sm = sm.add_constant(X_test)

        # Обучение модели
        model = sm.ZeroInflatedNegativeBinomialP(
            endog=y_train,
            exog=X_train_sm,
            exog_infl=X_train_sm,
            inflation='logit',
            exposure=exp_train
        )

        t0 = time()
        result = model.fit(method='bfgs', maxiter=100, disp=0, skip_hessian=True)
        t1 = time()

        # Предсказания
        y_pred = result.predict(exog=X_test_sm, exog_infl=X_test_sm, exposure=exp_test)
        mask = ~np.isnan(y_pred)

        if not np.any(mask):
            print(f"{feat:40s} | ❌ Все предсказания NaN")
            continue

        mse = mean_squared_error(y_test[mask], y_pred[mask])
        print(f"{feat:40s} | ✅ MSE={mse:.4f} | Time={t1-t0:.1f}s")

    except Exception as e:
        print(f"{feat:40s} | ❌ Failed: {str(e)}")


🔍 Начинаем поочерёдное добавление признаков к ядру...



C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:3937: RuntimeWarning: invalid value encountered in log
  a1 * np.log(a1) + y * np.log(mu) -
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:3938: RuntimeWarning: invalid value encountered in log
  (y + a1) * np.log(a2))
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:3974: RuntimeWarning: invalid value encountered in log
  dgterm = dgpart + np.log(a1 / a2) + 1 - a3 / a2
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:4329: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:3937: RuntimeWarning: invalid value encountered in multiply
  a1 * np.log(a1) + y * np.log(mu) -
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:3974: RuntimeWarning: divide by zero encountered in log
  dgterm = dgpart + np.log(

cat__category_Опрокидывание              | ✅ MSE=7337910485257367073234511594056526013100659837036789112308310582325827331914989568.0000 | Time=357.6s


KeyboardInterrupt: 

In [39]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Функция для обучения и оценки ZINB модели
def train_zinb_model(X_train, X_test, y_train, y_test, exposure_train, exposure_test, model_name):
    # Приводим к числам
    X_train_sm = sm.add_constant(X_train).astype(float)
    X_test_sm = sm.add_constant(X_test).astype(float)
    y_train = y_train.astype(float)
    y_test = y_test.astype(float)
    exposure_train = exposure_train.astype(float)
    exposure_test = exposure_test.astype(float)

    # Обучение модели
    zinb_model = sm.ZeroInflatedNegativeBinomialP(
        endog=y_train, 
        exog=X_train_sm, 
        exog_infl=X_train_sm, 
        inflation='logit', 
        exposure=exposure_train
    )
    zinb_result = zinb_model.fit(maxiter=50, disp=0)

    # Предсказания
    y_pred = zinb_result.predict(exog=X_test_sm, exog_infl=X_test_sm, exposure=exposure_test)

    # Удаляем NaN, если есть
    mask = ~np.isnan(y_pred)
    if not np.all(mask):
        print(f"[{model_name}] Предсказания содержат {np.sum(~mask)} NaN. Удаляем для MSE.")
        y_pred = y_pred[mask]
        y_test = y_test[mask]

    # MSE
    mse = mean_squared_error(y_test, y_pred)
    print(f"{model_name}:")
    print(f"  MSE: {mse:.4f}\n")

    return zinb_result, mse, X_train, X_test

# Подготовка данных
y = df_regression['remainder__dead_count']
exposure = df_regression['remainder__participants_count']

base_columns = [
    'cat__category_Наезд на велосипедиста', 'cat__category_Наезд на пешехода',
    'cat__category_Наезд на препятствие', 'cat__category_Опрокидывание',
    'cat__category_Падение пассажира', 'cat__category_Столкновение',
    'cat__category_Съезд с дороги', 'cat__light_В темное время суток, освещение включено',
    'cat__light_В темное время суток, освещение отсутствует', 'cat__light_Светлое время суток',
    'cat__light_Сумерки', 'cat__drivers_gender_all_female', 'cat__drivers_gender_mixed',
    'remainder__accident_id', 'remainder__avg_experience', 'remainder__avg_vehicle_age',
    'remainder__has_violating_pedestrian', 'remainder__adverse_weather'
]

# Комбинации признаков
combinations = [
    (['cat__day_of_week_Friday', 'cat__day_of_week_Saturday', 'cat__day_of_week_Sunday',
      'cat__day_of_week_Thursday', 'cat__day_of_week_Tuesday', 'cat__day_of_week_Wednesday'], 
     ['remainder__sin_time', 'remainder__cos_time'], 
     "ZINB: day_of_week + sin_time, cos_time"),
    (['cat__day_of_week_Friday', 'cat__day_of_week_Saturday', 'cat__day_of_week_Sunday',
      'cat__day_of_week_Thursday', 'cat__day_of_week_Tuesday', 'cat__day_of_week_Wednesday'], 
     ['cat__time_period_вечерний час пик', 'cat__time_period_дневная активность', 'cat__time_period_утренний час пик'], 
     "ZINB: day_of_week + time_period"),
    (['remainder__is_weekend'], 
     ['remainder__sin_time', 'remainder__cos_time'], 
     "ZINB: is_weekend + sin_time, cos_time"),
    (['remainder__is_weekend'], 
     ['cat__time_period_вечерний час пик', 'cat__time_period_дневная активность', 'cat__time_period_утренний час пик'], 
     "ZINB: is_weekend + time_period")
]

# Сравнение ZINB моделей
results = []
for day_cols, time_cols, name in combinations:
    X = df_regression[base_columns + day_cols + time_cols]
    X_train, X_test, y_train, y_test, exposure_train, exposure_test = train_test_split(
        X, y, exposure, test_size=0.2, random_state=42
    )
    
    result, mse, X_train, X_test = train_zinb_model(
        X_train, X_test, y_train, y_test, exposure_train, exposure_test, name
    )
    results.append((name, result, mse, X_train, X_test))

# Выбор лучшей ZINB модели по MSE
best_model = min(results, key=lambda x: x[2])
best_name, best_result, best_mse, X_train_best, X_test_best = best_model
print(f"Лучшая ZINB модель по MSE: {best_name}")
print(f"  MSE: {best_mse:.4f}\n")


C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:1567: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + exposure + offset)
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:1568: RuntimeWarning: invalid value encountered in multiply
  return -np.dot(L*X.T, X)
C:\Programs\Anaconda\Lib\site-packages\statsmodels\discrete\discrete_model.py:1478: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + offset + exposure)
C:\Programs\Anaconda\Lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


KeyboardInterrupt: 

In [ ]:

# Обучение ZIP модели с той же комбинацией
X_train_sm = sm.add_constant(X_train_best)
X_test_sm = sm.add_constant(X_test_best)

zip_model = sm.ZeroInflatedPoisson(
    endog=y_train, 
    exog=X_train_sm, 
    exog_infl=X_train_sm, 
    inflation='logit', 
    exposure=exposure_train
)
zip_result = zip_model.fit(maxiter=50, disp=0)

y_pred_zip = zip_result.predict(exog=X_test_sm, exog_infl=X_test_sm, exposure=exposure_test)
mse_zip = mean_squared_error(y_test, y_pred_zip)

print("Zero-Inflated Poisson с той же комбинацией:")
print(f"  MSE: {mse_zip:.4f}\n")

# Выбор итоговой модели
if mse_zip < best_mse:
    final_model = zip_result
    final_model_name = "Zero-Inflated Poisson"
    final_mse = mse_zip
else:
    final_model = best_result
    final_model_name = best_name
    final_mse = best_mse

print(f"Итоговая лучшая модель: {final_model_name}")
print(f"  MSE: {final_mse:.4f}\n")

# Интерпретация итоговой модели
print(f"Интерпретация лучшей модели ({final_model_name}):")
print(final_model.summary())

# Извлечение значимых переменных (p < 0.05)
coef_df = pd.DataFrame({
    'Variable': final_model.params.index,
    'Coefficient': final_model.params.values,
    'P-value': final_model.pvalues.values
})
significant_vars = coef_df[coef_df['P-value'] < 0.05]
print("\nЗначимые переменные (p < 0.05):")
print(significant_vars)

In [ ]:
2. **Выбор лучшей ZINB модели:**
Из 4 моделей выбрана одна с наименьшим значением MSE.
3. **Сравнение с Zero-Inflated Poisson (ZIP):**
Построена одна ZIP-модель с той же комбинацией признаков, что у лучшей ZINB модели.
Сравнение проведено по MSE, чтобы подтвердить превосходство ZINB, учитывающей overdispersion.
    
4. **Интерпретация итоговой модели**:
Итоговая модель (ZINB или ZIP с меньшим MSE) проанализирована через коэффициенты и p-значения.
Выделены значимые факторы (p < 0.05), влияющие на число погибших.

In [ ]:
### 3.3. Интерпретация

- Какие переменные увеличивают риск смертельных ДТП
- Величина коэффициентов и значимость
- Выбор лучшей модели (например, по AIC)

---

In [ ]:
## Не забыть кросс-валидацию
## 4. Модель 2: Дерево решений и случайный лес

### 4.1. Целевая переменная

Бинарная классификация: `is_fatal = dead_count > 0`

### 4.2. Построение моделей

```python
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Обучение
clf = RandomForestClassifier(n_estimators=100, max_depth=6)
clf.fit(X_train, y_train)

# Оценка
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))
```

### 4.3. Интерпретация

- Матрица ошибок
- Значимость признаков
- Что определяет наличие погибших

---

## 5. Модель 3: Экономический ущерб

### 5.1. Расчёт переменной

На основе методики из статьи N:  
> *“Оценка социально-экономического ущерба от ДТП в России”*, 2023  
> https://example.com/article-link ← впиши сюда реальную ссылку

```python
df["econ_damage"] = df["dead_count"] * 20_000_000 + df["injured_count"] * 2_000_000
df["log_cost"] = np.log1p(df["econ_damage"])
```

### 5.2. Модели

- Линейная регрессия
- Lasso (`LassoCV`)
- Gradient Boosting (`HistGradientBoostingRegressor`)

### 5.3. Интерпретация

- Как влияет каждый фактор на ущерб
- Можно ли интерпретировать в процентах (при логарифме — да)
- Визуализация важности признаков

---

## 6. Выводы

- Сравнение моделей (таблица с метриками)
- Практические интерпретации
- Возможности использования модели в политике/управлении
- Ограничения анализа (данные ГИБДД, пропуски, агрегирование)